In [17]:
import sys
sys.path.append('../MCMC_main/tools')
sys.path.append('../MCMC_main')
import uncertainty_DICT
import parameters_DICT
import geometry_binary
import os,glob
import pylab
import pyfits as pf
import numpy as np
import scipy
import matplotlib.pylab as plt
import scipy.constants as c
from PyAstronomy import pyasl
from scipy import interpolate
import csv
import pickle
import pandas as pd


In [19]:

def scale_intensity(wave_0,
                    wave_template,
                    intensity_template,
                    wave_spec,
                    intensity_spec,
                    norm=True,
                    wave_region=100e-10,
                    wave_0_d=100e-10):
    '''
    Returns the rescaled spectrum.
    norm=True if the spectrum is normalised.
    '''
    if norm==True:
        wave_min_a     = wave_0 - (wave_0_d + wave_region)
        wave_min_b     = wave_0 - (wave_0_d)
        wave_max_a     = wave_0 + (wave_0_d)
        wave_max_b     = wave_0 + (wave_0_d + wave_region)
        index_low  = np.argmax(intensity_template[np.where((wave_template > wave_min_a) & (wave_template < wave_min_b))])
        index_high = np.argmax(intensity_template[np.where((wave_template > wave_max_a) & (wave_template < wave_max_b))])
        I_low      = (intensity_template[np.where((wave_template > wave_min_a) & (wave_template < wave_min_b))])[index_low]
        I_high     = (intensity_template[np.where((wave_template > wave_max_a) & (wave_template < wave_max_b))])[index_high]
        wave_low   = (wave_template[np.where((wave_template > wave_min_a) & (wave_template < wave_min_b))])[index_low]
        wave_high  = (wave_template[np.where((wave_template > wave_max_a) & (wave_template < wave_max_b))])[index_high]
        gradient       = (I_high - I_low) / ( wave_high - wave_low)
        intercept      = I_high - gradient * wave_high
        scaling_interp = np.poly1d(np.array([gradient, intercept]))

        intensity_spec = intensity_spec * scaling_interp(wave_spec)

    # if norm==True:
    #     wave_min_a     = wave_0 - (wave_0_d + wave_region)
    #     wave_min_b     = wave_0 - (wave_0_d)
    #     wave_max_a     = wave_0 + (wave_0_d)
    #     wave_max_b     = wave_0 + (wave_0_d + wave_region)
    #     median_I_low   = np.median(intensity_template[np.where((wave_template > wave_min_a) & (wave_template < wave_min_b))])
    #     median_I_high  = np.median(intensity_template[np.where((wave_template > wave_max_a) & (wave_template < wave_max_b))])
    #     gradient       = (median_I_high - median_I_low) / ( 2 * wave_0_d + wave_region)
    #     intercept      = median_I_high - gradient *  (wave_0 + wave_0_d + 0.5 * wave_region)
    #     scaling_interp = np.poly1d(np.array([gradient, intercept]))
    #     intensity_spec = intensity_spec * scaling_interp(wave_spec)
    #     print(gradient, intercept)
    #     plt.plot(wave_spec, intensity_spec)
    #     plt.plot(wave_template, intensity_template)
    #     plt.show()
    return intensity_spec, scaling_interp



In [78]:
object_id = 'TW_Cam'
datafile  = object_id+'.dat'
listfile      = object_id+'.list'
line          = 'halpha'
v_range       = 2000
what          = 'observed'

central_wavelengths = {'halpha': 6562.8, 'hbeta': 4861.35, 'hgamma': 4340.47, 'hdelta': 4101.73}
central_wavelength  = central_wavelengths[line]
wave_min            = central_wavelength - (1000*v_range/c.speed_of_light) * central_wavelength
wave_max            = central_wavelength + (1000*v_range/c.speed_of_light) * central_wavelength

###### Read in the object specific and model parameters ########################
parameters = {}
InputDir   = '../input_data/'+object_id+'/'
InputFile  = datafile

###### Create the parameter dictionary with all jet, binary, and model parameters
parameters = parameters_DICT.read_parameters(InputDir+InputFile)
parameters['BINARY']['T_inf'] = geometry_binary.T0_to_IC(parameters['BINARY']['omega'],
                                                         parameters['BINARY']['ecc'],
                                                         parameters['BINARY']['period'],
                                                         parameters['BINARY']['T0'])

parameters['OTHER']['central_wavelength'] = central_wavelength

ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the object.dat file should be of type FLOAT
ValueError: The parameter 'None' of the 

In [79]:
def get_spectrum(filename, phase_value, parameters, v_range, bjd):


    datalist = glob.glob(filename)
    print(datalist)
    if len(datalist) > 0:

        # Read the data and the header is resp. 'spectrum' and 'header'
        spectrum = pf.getdata(datalist[0])
        header   = pf.getheader(datalist[0])

        crpix           = header.get('CRPIX1')-1
        crval           = header.get('CRVAL1')
        cdelt           = header.get('CDELT1')
        bvcor           = header.get('BVCOR')
        numberpoints    = len(spectrum)
        wavelengthbegin = (crval - crpix*cdelt)
        wavelengthend   = crval + (numberpoints-1)*cdelt
        wavelengths     = np.linspace(wavelengthbegin,wavelengthend,numberpoints)
        # Change from logarithmic to linear wavelength range
        wavelengthslin  = np.exp(wavelengths)
        print(wavelengthslin)
        
        spectrum_wave_min = min(range(len(wavelengthslin)), key = lambda j: abs(wavelengthslin[j]- (parameters['OTHER']['central_wavelength'] - 48)))
        spectrum_wave_max = min(range(len(wavelengthslin)), key = lambda j: abs(wavelengthslin[j]- (parameters['OTHER']['central_wavelength'] + 48)))
        
        print(spectrum_wave_min, spectrum_wave_max)
    
    return wavelengthslin, spectrum_wave_min

create_observed_spectra_file(object_id, parameters, InputDir)

0 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*578163_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00578163_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
[ 3764.06950941  3764.08908452  3764.10865974 ...,  9007.07389896
  9007.12074041  9007.16758211]
105485 108298
1 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*578786_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00578786_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
[ 3764.06950941  3764.08908452  3764.10865974 ...,  9007.07389896
  9007.12074041  9007.16758211]
105485 108298
2 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*580871_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00580871_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
[ 3764.08908452  3764.10865974  3764.12823506 ...,  9007.16758211
  9007.21442406  9007.26126625]
105484 108297
3 /lhome/dylanb/astronomy

105513 108326
26 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*834364_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00834364_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
[ 3764.01078468  3764.03035949  3764.0499344  ...,  9006.93337605
  9006.98021677  9007.02705774]
105488 108301
27 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*835486_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00835486_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
[ 3764.03035949  3764.0499344   3764.06950941 ...,  9006.93337605
  9006.98021677  9007.02705774]
105487 108300
28 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*842747_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00842747_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
[ 3764.06950941  3764.08908452  3764.10865974 ...,  9006.88653556
  9006.93337605  9006.98021677]
105485 108298
29 /lhom

In [69]:
def create_observed_spectra_file(object_id, parameters, InputDir):

    data_list = pd.read_csv(InputDir+object_id+'.list', delimiter=' ', names=['ID', 'full_date', 'object_id', 'bjd', 'bvcor', 'program_id', 'exposure_time', 'something', 'something_else', 'date'])
    
    phases    = ( (data_list.bjd - parameters['BINARY']['T_inf']) / parameters['BINARY']['period'] ) % 1

    filenames = [ '/lhome/dylanb/astronomy/objects/survey2020/'+object_id+'/*'+str(filenumber)+'_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits' for filenumber in data_list.ID]

    for (spectrum, filename) in enumerate(filenames):
        wavelength_spectrum, spectrum = get_spectrum(filename, phases[spectrum], parameters, v_range, data_list.bjd[spectrum])


    return

0 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*578163_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00578163_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
1 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*578786_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00578786_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
2 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*580871_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00580871_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
3 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*584659_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00584659_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
4 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*595925_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/o

0 0
38 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*904716_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00904716_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
39 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*907085_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00907085_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
40 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*908139_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00908139_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
41 /lhome/dylanb/astronomy/objects/survey2020/TW_Cam/*908725_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits
['/lhome/dylanb/astronomy/objects/survey2020/TW_Cam/00908725_HRF_OBJ_ext_CosmicsRemoved_log_merged_c.fits']
0 0
